In [49]:
#import mvpa2
import pandas as pd
import numpy as np
#import sklearn
#to manipulate data
from mvpa2.datasets import *
import scipy.io
from mvpa2.suite import *
if __debug__:
    debug.active.append('SMLR_')
import matplotlib as plt
%matplotlib inline

In [57]:
#makes a dict for task
matFile = scipy.io.loadmat('/Users/Alexis/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/mem/MSC02_parcel_corrmat.mat')
#convert to numpy array
task_FC=np.array(matFile['parcel_corrmat'])
#Index of upper triangle of matrix 
nrois=333
nsess=10
task_mask=np.triu_indices(nrois, 1)
task_corrlin=np.empty((nsess, int(nrois*(nrois-1)/2)))
count=0
#loop through all 10 days to reshape correlations into linear form
for sess in range(nsess):
    tmp=task_FC[:,:,sess]
    task_corrlin[count]=tmp[task_mask]
    count=count+1

#makes a dict for rest
matFile = scipy.io.loadmat('/Users/Alexis/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/rest/MSC02_parcel_corrmat.mat')
#convert to numpy array
rest_FC=np.array(matFile['parcel_corrmat'])
#Index of upper triangle of matrix 
nrois=333
nsess=10
rest_mask=np.triu_indices(nrois, 1)
rest_corrlin=np.empty((nsess, int(nrois*(nrois-1)/2)))
count=0
#loop through all 10 days to reshape correlations into linear form
for sess in range(nsess):
    tmp=rest_FC[:,:,sess]
    rest_corrlin[count]=tmp[rest_mask]
    count=count+1
    
#makes a dict for test task
matFile = scipy.io.loadmat('/Users/Alexis/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/mem/MSC03_parcel_corrmat.mat')
#convert to numpy array
motor_FC=np.array(matFile['parcel_corrmat'])
motor_FC=np.nan_to_num(motor_FC)
#Index of upper triangle of matrix 
nrois=333
nsess=10
test_mask=np.triu_indices(nrois, 1)
motor_corrlin=np.empty((nsess, int(nrois*(nrois-1)/2)))
count=0
#loop through all 10 days to reshape correlations into linear form
for sess in range(nsess):
    tmp=motor_FC[:,:,sess]
    motor_corrlin[count]=tmp[test_mask]
    count=count+1

#makes a dict for test task
matFile = scipy.io.loadmat('/Users/Alexis/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/rest/MSC03_parcel_corrmat.mat')
#convert to numpy array
r_FC=np.array(matFile['parcel_corrmat'])
r_FC=np.nan_to_num(r_FC)
#Index of upper triangle of matrix 
nrois=333
nsess=10
r_mask=np.triu_indices(nrois, 1)
r_corrlin=np.empty((nsess, int(nrois*(nrois-1)/2)))
count=0
#loop through all 10 days to reshape correlations into linear form
for sess in range(nsess):
    tmp=r_FC[:,:,sess]
    r_corrlin[count]=tmp[test_mask]
    count=count+1

#corrlin.shape
#task_ds=Dataset(corrlin)
#ds.shape
#ds.nfeatures
#ds.nsamples
#create a training dataset from the labeled features, targets 1:0 task:rest chunks=number of days
task_ds=dataset_wizard(samples=task_corrlin, targets=1, chunks=range(10))
rest_ds=dataset_wizard(samples=rest_corrlin, targets=0, chunks=range(10))
trainpat=vstack((task_ds, rest_ds))

#create pattern for testing dataset
test_ds=dataset_wizard(samples=motor_corrlin, targets=1, chunks=range(10))
r_df=dataset_wizard(samples=r_corrlin, targets=0, chunks=range(10))
testpat=vstack((test_ds, r_df))

In [3]:
#linear SVM
print "evaluating linear nu SVM classifier"
lsvm=LinearNuSVMC()
#save estimates
lsvm.ca.enable('estimates')
#train with known points
lsvm.train(trainpat)
#run predictions on test values
pre=lsvm.predict(test_ds.samples)
#calculate confusion matrix
lsvm_confusion=ConfusionMatrix(labels=trainpat.UT, targets=test_ds.targets,predictions=pre)
print "linear-SVM nu Percent Correct:\t%g%%" % \
    (lsvm_confusion.percent_correct)

evaluating linear SVM classifier
linear-SVM Percent Correct:	30%


//anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/misc/attrmap.py:153: FutureWarning: Conversion of the second argument of issubdtype from `str` to `str` is deprecated. In future, it will be treated as `np.string_ == np.dtype(str).type`.
  if not np.issubdtype(attr.dtype, str) and not self.mapnumeric:
//anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/clfs/libsvmc/svm.py:215: FutureWarning: Conversion of the second argument of issubdtype from `'c'` to `str` is deprecated. In future, it will be treated as `np.string_ == np.dtype('c').type`.
  if ( np.issubdtype(self.ca.trained_targets.dtype, 'c') or
//anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/clfs/libsvmc/svm.py:216: FutureWarning: Conversion of the second argument of issubdtype from `'U'` to `unicode` is deprecated. In future, it will be treated as `np.unicode_ == np.dtype('U').type`.
  np.issubdtype(self.ca.trained_targets.dtype, 'U') ):
//anaconda3/envs/ipykernel_py2/lib/python2.7/sit

In [4]:
#SVM
print "evaluating standard SVM classifier"
svm=SVM()
#save estimates
svm.ca.enable('estimates')
#train with known points
svm.train(trainpat)
#run predictions on test values
pre=svm.predict(test_ds.samples)
#calculate confusion matrix
svm_confusion=ConfusionMatrix(labels=trainpat.UT, targets=test_ds.targets,predictions=pre)
print "Standard SVM Percent Correct:\t%g%%" % \
    (svm_confusion.percent_correct)

evaluating SVM classifier
linear-SVM Percent Correct:	20%


In [5]:
#Linear SVM
print "evaluating linear SVM classifier"
lcsvmc=LinearCSVMC()
#save estimates
lcsvmc.ca.enable('estimates')
#train with known points
lcsvmc.train(trainpat)
#run predictions on test values
pre=lcsvmc.predict(test_ds.samples)
#calculate confusion matrix
lcsvmc_confusion=ConfusionMatrix(labels=trainpat.UT, targets=test_ds.targets,predictions=pre)
print "linear-SVM Percent Correct:\t%g%%" % \
    (lcsvmc_confusion.percent_correct)

evaluating linear SVM classifier
linear-SVM Percent Correct:	20%


In [6]:
#Linear SVM
print "evaluating C SVM classifier using radial basis function kernel"
RBfcsvmc=RbfCSVMC()
#save estimates
RBfcsvmc.ca.enable('estimates')
#train with known points
RBfcsvmc.train(trainpat)
#run predictions on test values
pre=RBfcsvmc.predict(test_ds.samples)
#calculate confusion matrix
RBfcsvmc_confusion=ConfusionMatrix(labels=trainpat.UT, targets=test_ds.targets,predictions=pre)
print "linear-SVM Percent Correct:\t%g%%" % \
    (RBfcsvmc_confusion.percent_correct)

evaluating C SVM classifier using radial basis function kernel
 * Please note: warnings are printed only once, but underlying problem might occur many times *
linear-SVM Percent Correct:	0%


//anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/clfs/transerror.py:685: RuntimeWarning: invalid value encountered in divide
  stats['PPV'] = stats['TP'] / (1.0*stats["P'"])
//anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/clfs/transerror.py:686: RuntimeWarning: invalid value encountered in divide
  stats['NPV'] = stats['TN'] / (1.0*stats["N'"])
//anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/clfs/transerror.py:687: RuntimeWarning: invalid value encountered in divide
  stats['FDR'] = stats['FP'] / (1.0*stats["P'"])


In [58]:
#create objects holding linear svm, cross validation scheme==leave one out cross validation, accuracy
cv = CrossValidation(LinearCSVMC(), NFoldPartitioner(), #Nfold is leave one out; selects one chunk at a time chunk being day
                     enable_ca=['stats'], 
                     errorfx=lambda p, t: np.mean(p == t))#reports accuracy instead of error
#classifier predictions and the target values stored in the dataset

In [68]:
#res = cv(trainpat)
#print "accuracy of linear svmc"
#np.mean(res.samples) - (1.0 / len(trainpat.UT)) # % above chance accuracy

In [69]:
pre=cv.train(trainpat)
#call the classifier directly to test the new dataset
#this computes the error so lower values represent more accurate classification


In [70]:
err=cv(testpat)

In [73]:
pre #prediction accuracy

In [62]:
np.mean(res)

0.95

In [63]:
res.samples #results of all cross validated folds. 
#Accuracy and a sample per fold 
#looks like it was able to classify across 9 folds

array([[1. ],
       [1. ],
       [1. ],
       [1. ],
       [1. ],
       [1. ],
       [1. ],
       [1. ],
       [1. ],
       [0.5]])

In [74]:
print cv.ca.stats.as_string(description=True)

----------.
predictions\targets   0    1
            `------  ---  --- P' N' FP FN PPV NPV TPR SPC FDR MCC F1 AUC
         0           10    0  10 10  0  0  1   1   1   1   0   1   1  1
         1            0   10  10 10  0  0  1   1   1   1   0   1   1  1
Per target:          ---  ---
         P           10   10
         N           10   10
         TP          10   10
         TN          10   10
Summary \ Means:     ---  --- 10 10  0  0  1   1   1   1   0   1   1  1
       CHI^2         20  p=0.00017
        ACC           1
        ACC%         100
     # of sets       10   ACC(i) = 1+0*i p=1 r=0 r^2=0

Statistics computed in 1-vs-rest fashion per each target.
Abbreviations (for details see http://en.wikipedia.org/wiki/ROC_curve):
 TP : true positive (AKA hit)
 TN : true negative (AKA correct rejection)
 FP : false positive (AKA false alarm, Type I error)
 FN : false negative (AKA miss, Type II error)
 TPR: true positive rate (AKA hit rate, recall, sensitivity)
      TPR = TP / P 

In [75]:
print cv.ca.stats.matrix

[[10  0]
 [ 0 10]]
